In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [ ]:
import os
import json
import numpy as np
import pickle
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
import contextily as cx
import folium
from tqdm import trange, tqdm

from polygon_compare import *

In [ ]:
# ms house data
ordu_house = os.path.join("data","clipped_house","city_house.pkl")
johannesburg_house = os.path.join("data","clipped_house","Johannesburg_house.pkl")
cairo_house = os.path.join("data","clipped_house","Cairo_house.pkl")
manisa_house = os.path.join("data","clipped_house","Manisa_house.pkl")
adana_house = os.path.join("data","clipped_house","Adana_house.pkl")
trabzon_house = os.path.join("data","clipped_house","Trabzon_house.pkl")

In [ ]:
# osm house data
city_osm = os.path.join("data","OSM","city.pkl")


In [ ]:
# osmb house data
ordu_osmb = os.path.join("data","OSMB","Ordu.pkl")


In [ ]:
df_ms = pickle.load(open(ordu_house,"rb"))
df_osm = pickle.load(open(ordu_osm,"rb"))
df_osmb = pickle.load(open(ordu_osmb,"rb"))

In [ ]:
df_osmb = df_osmb.set_crs('EPSG:4326')
df_ms = df_ms.to_crs('EPSG:4326')
df_ms = df_ms[df_ms["area"]>30]
df_ms

In [ ]:
df_osm_b = df_osm[df_osm["type"]=="way"]
df_osm_b

In [ ]:
cities = [
    # 'Ordu',
    # 'Manisa',
    # 'Adana',
    
    # 'Johannesburg',
    # 'Cairo',
    # 'Trabzon',
]

for c in cities:
    city_osm = os.path.join("data","OSM",f"{c}.pkl")
    df_osm = pickle.load(open(city_osm,"rb"))
    df_osm_b = df_osm[df_osm["type"]=="way"]
    
    city_osmb = os.path.join("data","OSMB",f"{c}.pkl")
    df_osmb = pickle.load(open(city_osmb,"rb"))
    df_osmb = df_osmb.set_crs('EPSG:4326')
    
    print(df_osmb.shape)

    df = compare_geodf(df_osmb, df_osm_b, match_one=True)
    # df.plot()
    df.to_pickle(f'{c}_combined_osm.pkl')

In [ ]:
cities = [
    # 'Ordu',
    'Manisa',
    'Adana',
    'Cairo',
    
    # 'Trabzon',
    # 'Johannesburg',
]

for c in cities:
    city_osm_combined = os.path.join("data","OSM_combined",f"{c}_combined_osm.pkl")
    city_osm_combined = pickle.load(open(city_osm_combined,"rb"))
    
    city_house = os.path.join("data","clipped_house",f"{c}_house.pkl")
    df_ms = pickle.load(open(city_house,"rb"))
    df_ms = df_ms[df_ms["area"]>30]
    df_ms = df_ms.to_crs('EPSG:4326')

    print(city_osm_combined.shape)
    print(df_ms.shape)

    df = compare_geodf2(city_osm_combined, df_ms, match_one=False)
    # df.plot()
    df.to_pickle(f'{c}_combined_osm_ms.pkl')
    

In [ ]:
df.to_file('ordu_house.shp')


In [ ]:
m = df_osm_b.explore(
    color="green",
    name="OSM"
)

df_osmb.explore(
     m=m,
    column="height",
    name="OSMB"
)

# df_ms.explore(
#      m=m,
#     color="blue",
#     name="MS"
# )

# combined_polygons.explore(
#      m=m,
#     color="red",
#     name="MS2"
# )

folium.TileLayer(control=True).add_to(m)  # use folium to add alternative tiles
folium.LayerControl().add_to(m)  # use folium to add layer control
m

---

In [ ]:
from utils import *
from create_polygon import Create_random_polygon

In [ ]:
def make_grid(polygon, edge_size):
    """
    polygon : shapely.geometry
    edge_size : length of the grid cell
    """
    from itertools import product
    import numpy as np
    import geopandas as gpd
    
    bounds = polygon.bounds    
    x_coords = np.arange(bounds[0] + edge_size/2, bounds[2], edge_size)
    y_coords = np.arange(bounds[1] + edge_size/2, bounds[3], edge_size)
    combinations = np.array(list(product(x_coords, y_coords)))
    
    squares = gpd.points_from_xy(combinations[:, 0], combinations[:, 1]).buffer(edge_size / 2, cap_style=3)
    return gpd.GeoSeries(squares[squares.intersects(polygon)])

In [ ]:
c="Cairo"


In [ ]:
ordu = pickle.load(open(os.path.join("data","Cairo_v1.pkl"),"rb"))


In [ ]:


city_house = os.path.join("data","clipped_house",f"{c}_house.pkl")
df_ms = pickle.load(open(city_house,"rb"))
df_ms = df_ms[df_ms["area"]>30]
df_ms = df_ms.to_crs('EPSG:4326')

In [ ]:
df_ms.total_bounds


In [ ]:
box = [30.8, 29.7, 31.8, 30.4]
box = create_polygon_from_bounds(box)
box.bounds

In [ ]:
df = ordu.inner_box.region
p = df.dissolve()

In [ ]:
boxes = make_grid(box,0.05)
boxes.plot(edgecolor='black',)

In [ ]:
df_ms.clip(boxes[0])


In [ ]:
city_osm_combined = os.path.join("data","OSM_combined",f"{c}_combined_osm.pkl")
city_osm_combined = pickle.load(open(city_osm_combined,"rb"))
buffered = city_osm_combined.buffer(0)

In [ ]:
for (i, b) in enumerate(boxes):
    index = buffered.clip(b.buffer(0)).index
    city_osm_combined.loc[index,"box_index"] = i

In [ ]:
city_osm_combined.loc[index,"box_index"] = i


In [ ]:
city_osm_combined["box_index"].unique()


In [ ]:
c="Cairo"
df = os.path.join("data","OSM",f"{c}.pkl")
df = pickle.load(open(df,"rb"))
df= df.to_crs("EPSG:3395")

In [ ]:
invalid = df[~df.is_valid].index


In [ ]:
# res = reorder_points_90(df.loc[invalid[0]]["geometry"])
df.loc[df["id"]==236879520, "geometry"] = reorder_points_90(df.loc[df["id"]==236879520,"geometry"].iloc[0])
df[df["id"]==236879520]

In [ ]:
res = reorder_points_90(df.loc[invalid[1]]["geometry"])
res

In [ ]:
df= df.to_crs("EPSG:4326")
df[df["id"]==236879520]


In [ ]:
reorder_points_90(df.loc[invalid[0]]["geometry"]).is_valid


In [ ]:
df.loc[invalid[0:1]].explore()
df.loc[invalid].explore()

In [ ]:
new_p = []
for index, r in df[~df.is_valid].iterrows():
    print(index)
    # df.loc[index, "geometry"] = reorder_points(r["geometry"])
    # df.loc[index, "geometry"] = r["geometry"].exterior.convex_hull
    # temp = Create_random_polygon(array=list(r["geometry"].exterior.coords))
    df.loc[index, "geometry"] = reorder_points_90(r["geometry"])

    polygon_array = temp.main()
    df.loc[index, "geometry"] =  Polygon(polygon_array)

In [ ]:
df= df.to_crs("EPSG:4326")
df.loc[invalid].explore()

In [ ]:
df.loc[invalid]

In [ ]:
df.loc[[40814,40818, 40823, 40834, 40836]].explore()


In [ ]:
c="Cairo"
df = os.path.join("data","OSM_combined",f"{c}_combined_osm.pkl")
df = pickle.load(open(df,"rb"))
df[df["geometry"].isna()]


In [ ]:
df = os.path.join("data","OSMB",f"{c}.pkl")
df = pickle.load(open(df,"rb"))
df

In [ ]:
df[df["geometry"].isna()]


In [ ]:
c="Ordu"
df = os.path.join("data","ml",f"{c}.pkl")
df = pickle.load(open(df,"rb"))


In [ ]:
df[df["geometry"].isna()]


In [ ]:
c="Ordu"
df = os.path.join("data","clipped_house",f"{c}_house.pkl")
df = pickle.load(open(df,"rb"))
df

In [ ]:
df.loc[[17853760]]["geometry"].plot()

In [ ]:
paths = [
    os.path.join("data","OSM"),
    os.path.join("data","OSMB"),
    os.path.join("data","clipped_house"),
]

df = pickle.load(open(os.path.join(paths[2],f"{c}_labeled.pkl"),"rb"))

In [ ]:
df.crs
df[df["box_index"].isna()]
